<a href="https://colab.research.google.com/github/tj191073-droid/tj191073/blob/main/fishnet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://huggingface.co/datasets/imageomics/fish-vista

Cloning into 'fish-vista'...
remote: Enumerating objects: 80157, done.
remote: Total 80157 (delta 0), reused 0 (delta 0), pack-reused 80157 (from 1)
Receiving objects: 100% (80157/80157), 212.04 MiB | 19.05 MiB/s, done.
Resolving deltas: 100% (104/104), done.
Updating files: 100% (75833/75833), done.
Filtering content: 100% (75730/75730), 11.04 GiB | 21.80 MiB/s, done.


In [2]:
%cd fish-vista
!git lfs install
!git lfs pull

/content/fish-vista
Updated git hooks.
Git LFS initialized.


In [3]:
import os
print(os.path.exists("/content/fish-vista/classification_train.csv"))

True


In [10]:
import pandas as pd
import os

# 1. 加载原始数据
df = pd.read_csv("/content/fish-vista/classification_train.csv", low_memory=False)

# 2. 统计每个鱼科下的图像数量（按物种聚合）
grouped = df.groupby(["family", "standardized_species"]).size().reset_index(name='count')

# 3. 筛选出图像数量 >= 30 的物种
grouped = grouped[grouped['count'] >= 30]

# 4. 每个鱼科最多随机选 2 个物种（鱼类），最多保留 5 个鱼科
selected_families = grouped.groupby("family")['count'].sum().sort_values(ascending=False).head(5).index.tolist()
filtered = grouped[grouped['family'].isin(selected_families)]

# 5. 每个鱼科最多选 2 个物种
selected_species = (
    filtered.groupby("family")
    .apply(lambda x: x.sample(n=min(len(x), 2), random_state=42))
    .reset_index(drop=True)
)

# 6. 从原始 df 中筛选图像记录
sub_df = df[df['standardized_species'].isin(selected_species['standardized_species'])]

# 7. 每个物种最多取 30 张
subset = sub_df.groupby('standardized_species', group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), 30), random_state=42)
).reset_index(drop=True)

# 8. 构建完整图像路径
def resolve_path(file_name):
    try:
        chunk = file_name.split('/')[1].split('_')[1]
        return os.path.join("/content/fish-vista/Images", f"chunk_{chunk}", os.path.basename(file_name))
    except:
        return None

subset['image_path'] = subset['file_name'].apply(resolve_path)
subset = subset[subset['image_path'].notnull()]
subset['image_exists'] = subset['image_path'].apply(lambda x: os.path.exists(x))
subset = subset[subset['image_exists']].reset_index(drop=True)

# 9. 展示摘要
print(f"✅ 最终子集包含 {subset['family'].nunique()} 个鱼科，{subset['standardized_species'].nunique()} 个物种，{len(subset)} 张图像")
print(subset[['family', 'standardized_species']].value_counts())

✅ 最终子集包含 8 个鱼科，10 个物种，299 张图像
family         standardized_species
Centrarchidae  lepomis megalotis       30
               lepomis miniatus        30
Cottidae       cottus perplexus        30
Cyprinidae     notropis rubellus       30
Esocidae       esox americanus         30
Ictaluridae    noturus exilis          30
Esocidae       esox lucius             30
Cottidae       cottus carolinae        29
Cyprinidae     notropis telescopus     28
Ictaluridae    noturus eleutherus      28
ictaluridae    noturus eleutherus       2
cottidae       cottus carolinae         1
cyprinidae     notropis telescopus      1
Name: count, dtype: int64


/tmp/ipython-input-10-3213855260.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min(len(x), 2), random_state=42))
/tmp/ipython-input-10-3213855260.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  subset = sub_df.groupby('standardized_species', group_keys=False).apply(


In [11]:
import shutil
from sklearn.model_selection import train_test_split

# 指定输出目录
base_dir = "/content/fish_dataset_mini"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")

# 清空旧目录
if os.path.exists(base_dir):
    shutil.rmtree(base_dir)

# 创建 train/val 目录
for d in [train_dir, val_dir]:
    os.makedirs(d, exist_ok=True)

# 按类别划分图像
for cls in subset["standardized_species"].unique():
    cls_df = subset[subset["standardized_species"] == cls]
    image_paths = cls_df["image_path"].tolist()
    image_paths = [p for p in image_paths if isinstance(p, str)]  # 防止 None

    train_imgs, val_imgs = train_test_split(image_paths, test_size=0.2, random_state=42)

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for src in train_imgs:
        shutil.copy(src, os.path.join(train_dir, cls, os.path.basename(src)))
    for src in val_imgs:
        shutil.copy(src, os.path.join(val_dir, cls, os.path.basename(src)))

print("✅ 图像划分完成（train/val）")

✅ 图像划分完成（train/val）


In [13]:
import os
import torch
from torch import nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# 设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据路径
data_dir = "/content/fish_dataset_mini"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
num_classes = len(os.listdir(train_dir))

# 图像增强与加载
image_size = 128
batch_size = 8

train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

train_data = datasets.ImageFolder(train_dir, transform=train_transform)
val_data = datasets.ImageFolder(val_dir, transform=val_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)

# 模型结构：MobileNetV2 + 分类头
model = models.mobilenet_v2(weights="MobileNet_V2_Weights.DEFAULT")
model.classifier[1] = nn.Linear(model.last_channel, num_classes)
model = model.to(device)

# 损失和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 模型训练
best_val_acc = 0.0
save_path = "best_model.pth"

for epoch in range(1, 21):  # 训练 20 轮
    model.train()
    correct, total = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()
    train_acc = correct / total

    # 验证
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    val_acc = correct / total

    print(f"Epoch {epoch}: Train Acc = {train_acc:.4f}, Val Acc = {val_acc:.4f}")

    # 保存最优模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print(f"✅ Saved best model (val acc = {val_acc:.4f})")

print("🎯 训练完成。最优模型保存在 best_model.pth")

Epoch 1: Train Acc = 0.4770, Val Acc = 0.7333
✅ Saved best model (val acc = 0.7333)
Epoch 2: Train Acc = 0.6946, Val Acc = 0.7333
Epoch 3: Train Acc = 0.7992, Val Acc = 0.8000
✅ Saved best model (val acc = 0.8000)
Epoch 4: Train Acc = 0.8536, Val Acc = 0.8500
✅ Saved best model (val acc = 0.8500)
Epoch 5: Train Acc = 0.8870, Val Acc = 0.8167
Epoch 6: Train Acc = 0.8661, Val Acc = 0.8333
Epoch 7: Train Acc = 0.8703, Val Acc = 0.7333
Epoch 8: Train Acc = 0.8787, Val Acc = 0.7500
Epoch 9: Train Acc = 0.8117, Val Acc = 0.7333
Epoch 10: Train Acc = 0.8703, Val Acc = 0.7167
Epoch 11: Train Acc = 0.9498, Val Acc = 0.7833
Epoch 12: Train Acc = 0.9414, Val Acc = 0.8167
Epoch 13: Train Acc = 0.9331, Val Acc = 0.8000
Epoch 14: Train Acc = 0.9456, Val Acc = 0.7333
Epoch 15: Train Acc = 0.9414, Val Acc = 0.8167
Epoch 16: Train Acc = 0.9121, Val Acc = 0.7167
Epoch 17: Train Acc = 0.9372, Val Acc = 0.8333
Epoch 18: Train Acc = 0.9331, Val Acc = 0.7500
Epoch 19: Train Acc = 0.9331, Val Acc = 0.8667
✅ 

In [25]:
from google.colab import files
from PIL import Image
from torchvision import transforms
import torch
import os

# 上传图片
uploaded = files.upload()
img_path = list(uploaded.keys())[0]
print(f"✅ 上传成功：{img_path}")

Saving 107737_dor_FMNH_FZ.jpg to 107737_dor_FMNH_FZ.jpg
✅ 上传成功：107737_dor_FMNH_FZ.jpg


In [24]:
# 图像预处理（要和训练时保持一致）
predict_tf = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

# 预测函数
def predict_image(model, img_path, class_names):
    model.eval()
    img = Image.open(img_path).convert("RGB")
    input_tensor = predict_tf(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        prob = torch.nn.functional.softmax(output[0], dim=0)
        pred_idx = torch.argmax(prob).item()
        pred_class = class_names[pred_idx]
        confidence = prob[pred_idx].item()

    print(f"✅ 预测类别：{pred_class}，置信度：{confidence:.2f}")
    return pred_class, confidence

In [26]:
# 类别名来自训练集
class_names = train_data.classes

# 调用预测
predict_image(model, img_path, class_names)

✅ 预测类别：cottus carolinae，置信度：0.97


('cottus carolinae', 0.9741412401199341)